# Introduction

Create a notebook on how to search through a specific airline website like Expedia or Southwest, find the best flight rates, and send an alert to your email on a timed basis. This is a nice project done by Aref. On top of emailing, we can store the data into a format that is usable for analysis purposes or historical view.

# Instructions

1. Connect Python to a web browser and access the airline website
2. Choose flights (roundtrip, one way, multi-city)
3. Select origin (city or airport of departure)
4. Select destination (city or airport of arrival)
5. Select departing and returning dates
6. Structure the data in a structured format for exploratory data analysis (optional)
7. Connect to your email and push an alert every hour

# What do I hope to achieve from this?

Develop a stronger understanding in working with other types of projects besides machine learning and data analysis. Webscraping is a process of extracting information and data from a website, transforming the information on a webpage into structured data for further analysis. Automation is a powerful skill to have and learning to do so with new data instead of extracting it from CSV files or SQL can be very handy in the future.

1. Automate my flight searches from OC to SJ
2. Initializing a new project and working from concept to product
3. Deploy to Production (optional) 

## Import libraries

In [3]:
# Data
import pandas as pd

# Web
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Misc
import time
import datetime
import os
from PIL import Image

# Email
import smtplib
from email.mime.multipart import MIMEMultipart

### Connect to the web

In [4]:
# Opens empty web browser of your choice
# browser = webdriver.Firefox()
browser = webdriver.Chrome()

Inspect the element we want to start working on and check the tags related to it.

It has a 'label' tag with 'id = flight-type-roundtrip-label-hp-flight'. This is the tag we will use to device on the ticket type.

Here are the tags below for each different ticket type id. Store the tags and ids.

_Note: You can right click the element and copy the XPATH. The roundtrip result would be copied and it will be something like this: "//*[@id="flight-type-roundtrip-label-hp-flight"]"_

In [5]:
# Set ticket types
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way_ticket = "//label[@id='flight-type-one-way-label-hp-flight']"

### Define a function to choose a roundtrip or one way ticket.

In [6]:
# Define a new function
def choose_ticket(ticket):
    '''
    Try to click the button to choose the ticket type
    '''
    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        time.sleep(1.0)
        ticket_type.click()
    except Exception as e:
        pass

### Define a function to choose departure the airport/city

The function will perform the following steps:
1. Type in the departure airport/city
2. Choose the first value from the drop-down menu

In [7]:
# Define a new function
def choose_departure_city(airport):
    '''
    Click flying from box, input airport, and select the first choice from the drop-down menu
    '''
    # Find departure box using the element's tags and attributes
    fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
    time.sleep(1.5)
    
    # Clear any value written in the field
    fly_from.clear()
    time.sleep(2.0)
    
    # Type in the airport that will be passed into the function using .sendkeys()
    fly_from.send_keys('  ' + airport)
    time.sleep(1.0)
    
    # Get XPATH from the dropdown after passing the airport value
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    
    # Click the first value
    first_item.click()

### Define a function to choose arrival the airport/city

The function will perform the following steps:
1. Type in the arrival airport/city
2. Choose the first value from the drop-down menu

In [8]:
# Define a new function
def choose_arrival_city(airport):
    '''
    Click flying from box, input airport, and select the first choice from the drop-down menu
    '''
    # Find departure box using the element's tags and attributes
    fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
    time.sleep(1.5)
    
    # Clear any value written in the field
    fly_to.clear()
    time.sleep(2.0)
    
    # Type in the airport that will be passed into the function using .sendkeys()
    fly_to.send_keys('  ' + airport)
    time.sleep(1.0)
    
    # Get XPATH from the dropdown after passing the airport value
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    
    # Click the first value
    first_item.click()

### Define a function to choose the departure date

The function will perform the following steps:
1. Find the date element on the web page
2. Clear every value
3. Type in the date in this specific format: mm/dd/yyyy

In [9]:
# Define a new function
def choose_departure_date(month, day, year):
    '''
    Function will select each individual part of the departure date
    '''
    # Find date using the element's tags and attributes
    dep_date = browser.find_element_by_xpath("//input[@id='flight-departing-hp-flight']")
    
    # Clear any value in the field
    dep_date.clear()
    
    # Type in the date that will be passed into the function using .sendkeys() 
    dep_date.send_keys(month + '/' + day + '/' + year)

### Define a function to choose the return date

In [10]:
# Define a new function
def choose_return_date(month, day, year):
    '''
    Function will select each individual part of the return date
    '''
    # Find date using the element's tag and attributes
    ret_date = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")
    
#     # Clear any value in the field
#     ret_date.clear()
    
#     # Type in the date that will be passed into the function
#     ret_date.send_keys(month + '/' + day + '/' + year)

    for i in range(11):
        ret_date.send_keys(Keys.BACKSPACE)
    ret_date.send_keys(month + '/' + day + '/' + year)

### Search

In [12]:
# Define a function that will click the search button
def search():
    '''
    Function will now press the select button after inputting all of the values and load the next page
    '''
    # Find search element's tag and attributes
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action gcw-submit']")
    
    # Click the button and wait for the page to load
    search.click()
    time.sleep(10)
    
    # Result
    print('Search results are loaded.')

### Get the data

Now that we have loaded the search results, we can proceed to the next steps in gathering the data so we could use it. Here are the steps in the following cell:
1. Create an empty df
2. Create variables for all of the flight attributes and store it in a list
3. Find all the elements for an attribute
4. Store the elements in the related variable created in the list before
5. Create lists side by side as columns into the dataframe
6. Save the dataframe to an Excel sheet

In [ ]:
# Create an empty df
df = pd.DataFrame()

# Define  function to gather the data calling our functions from before
def get_data():
    global df
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list
    
    ## Get elements
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [x.text for x in dep_times]
    
    # Departure times
    
    # Arrival Times
    
    # Airline Name
    
    # Prices
    
    # Durations
    
    # Stops
    
    # Layovers
    
    
    ## Create the data in a structured format
    
    
    
    
    
    
    
    
    
    
    
    

In [ ]:
https://www.learndatasci.com/tutorials/ultimate-guide-web-scraping-w-python-requests-and-beautifulsoup/